<a href="https://colab.research.google.com/github/quangly/MovieRecommendation/blob/main/MovieRecommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Description: Build a movie recommendation engine using Python

In [3]:
# Import the libraries
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [142]:
#Load the data
from google.colab import files
uploaded = files.upload()

Saving movie_data-new.csv to movie_data-new.csv


In [143]:
df = pd.read_csv("movie_data.csv")
df = pd.read_csv("movie_data-new.csv")
df['Movie_id'] = range(0,1000)
# shift column 'Name' to first position
first_column = df.pop('Movie_id')
  
# insert column using insert(position,column_name,
# first_column) function
df.insert(0, 'Movie_id', first_column)
#rename column Category to Genre
df = df.rename(columns={"Category": "Genre"})
df.head(3)



,Movie_id,Poster_Link,Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
0,0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,"28,341,469"
1,1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411"
2,2,https://m.media-amazon.com/images/M/MV5BMTMxNT...,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444"


In [144]:
#get count of numbr of row/smovies in the data set and number of columns
df.shape

(1000, 17)

In [148]:
#create list of important columns for the recommendation engine
# columns = ['Director', 'Star','Genre','Title']
columns = ['Director', 'Star1','Star2','Star3','Star4','Genre','Title']

In [149]:
#show the data
df[columns].head(3)


,Director,Star1,Star2,Star3,Star4,Genre,Title
0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,Drama,The Shawshank Redemption
1,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,"Crime, Drama",The Godfather
2,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,"Action, Crime, Drama",The Dark Knight


In [150]:
#check for any missing values in the import columns
# df[columns].isnull().values.any()
df[columns].isnull().values.any()
df.columns[df.isna().any()].tolist()
# df.loc[:, df.isnull().any()]

['Certificate', 'Meta_score', 'Gross']

In [165]:
# create a function to combine the values of the import columns into a single string
def get_important_features(data):
  important_features = []
  for i in range(0,data.shape[0]):
    important_features.append(data['Star1'][i] + \
                              ' ' + data['Star2'][i] + \
                              ' ' + data['Star3'][i] + \
                              ' ' + data['Star4'][i] + \
                              ' ' +data['Director'][i] + \
                              ' ' +data['Title'][i] + \
                              ' ' +data['Genre'][i])
  
  return important_features



In [166]:
#create a column to hold the combined strings
df['important_features'] = get_important_features(df)
# #show the data
# pd.set_option('display.max_rows', None)
# # df['important_features'].unique()
df.head(3)

,Movie_id,Poster_Link,Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross,important_features
0,0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,"28,341,469",Tim Robbins Morgan Freeman Bob Gunton William ...
1,1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411",Marlon Brando Al Pacino James Caan Diane Keato...
2,2,https://m.media-amazon.com/images/M/MV5BMTMxNT...,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444",Christian Bale Heath Ledger Aaron Eckhart Mich...


In [167]:
# convert the text to a matrix of token counts
cm = CountVectorizer().fit_transform(df['important_features'])

In [168]:
#get the cosine simimlarity matrix from the count matrix
cs = cosine_similarity(cm)
#Print the cosine similarity matrix - values 0 to 1. 1 = 100% similar to a movie
#each row is a movie and each column is a movie.
#first row/column is 1 b/c it's a movie similar to itself
print(cs)

[[1.         0.13801311 0.13363062 ... 0.06482037 0.14824986 0.06681531]
 [0.13801311 1.         0.19364917 ... 0.06262243 0.07161149 0.12909944]
 [0.13363062 0.19364917 1.         ... 0.06063391 0.06933752 0.125     ]
 ...
 [0.06482037 0.06262243 0.06063391 ... 1.         0.13453456 0.        ]
 [0.14824986 0.07161149 0.06933752 ... 0.13453456 1.         0.13867505]
 [0.06681531 0.12909944 0.125      ... 0.         0.13867505 1.        ]]


In [169]:
#get the shape of the cosine similarity matrix
cs.shape

(1000, 1000)

In [176]:
#get the title of the movie that user likes
title = "The Dark Knight"
#find the movies id
movie_id = df[df.Title == title]['Movie_id'].values[0]

In [183]:
#create a list of enumerations for the similarity score [(movie_id, similarity score), (..)]
scores = list(enumerate(cs[movie_id]))

#Sort the list x[1] = similarity score descending score
sorted_scores = sorted(scores, key = lambda x:x[1], reverse = True)
#get every element exempt first element
sorted_scores = sorted_scores[1:]


In [181]:
print(sorted_scores)

[(2, 1.0), (36, 0.5), (63, 0.5), (155, 0.4677071733467426), (774, 0.3125), (832, 0.30316953129541624), (819, 0.2795084971874737), (603, 0.2727723627949905), (718, 0.2727723627949905), (821, 0.2727723627949905), (573, 0.2672612419124244), (217, 0.2581988897471611), (614, 0.2581988897471611), (649, 0.2581988897471611), (953, 0.2581988897471611), (968, 0.2581988897471611), (983, 0.2581988897471611), (241, 0.25), (600, 0.25), (655, 0.25), (663, 0.25), (778, 0.25), (782, 0.25), (850, 0.25), (901, 0.25), (5, 0.24659848095803594), (10, 0.24659848095803594), (25, 0.24253562503633297), (384, 0.24253562503633297), (730, 0.24253562503633297), (852, 0.24253562503633297), (13, 0.24056261216234406), (278, 0.23570226039551587), (642, 0.23570226039551587), (661, 0.22941573387056174), (692, 0.22941573387056174), (28, 0.22360679774997896), (695, 0.22360679774997896), (442, 0.2182178902359924), (546, 0.2182178902359924), (612, 0.2182178902359924), (872, 0.2182178902359924), (55, 0.2165063509461097), (331

In [186]:
#create a loop to print the first 7 similar movies
j = 0
print('The 7 most recommended movies to', title, 'are:" \n')
for item in sorted_scores[:7]:
  movie_title = df[df.Movie_id == item[0]]['Title'].values[0]
  print(j+1, movie_title)
  j = j+1

The 7 most recommended movies to The Dark Knight are:" 

1 The Prestige
2 The Dark Knight Rises
3 Batman Begins
4 3:10 to Yuma
5 Empire of the Sun
6 The Muppet Christmas Carol
7 The Girl with the Dragon Tattoo
